# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.015140752916174, 5.641055696800138, 2.8442276243273326, 5.566908634271693, 3.893542190177791, 5.154014998651131, 4.731958879172958, 4.892423031457026, 4.330704622709736, 5.688413532390584]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [4]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[9.83429584962833, 8.156877448815841, 7.821766218045859, 8.884732248835702, 9.835974558206726, 7.921689632480854, 8.320755084537545, 6.236036587564709, 5.880260520334039, 8.607218202703514]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [5]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[8.607218202703514, 5.688413532390584, 5.880260520334039, 6.236036587564709, 4.330704622709736, 8.320755084537545, 4.892423031457026, 7.921689632480854, 9.835974558206726, 8.884732248835702]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Mean, variance

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [15]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.688413532390584, 5.880260520334039, 6.236036587564709, 4.330704622709736, 4.892423031457026]
[8.607218202703514, 8.320755084537545, 7.921689632480854, 9.835974558206726, 8.884732248835702]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.405567658891219,  mean_2 = 8.714073945352869
var_1 = 0.4827323395132675,  var_2 = 0.4163552033930724


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [21]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0]*prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1]*prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  8.607218202703514
probability of observing that point if it came from cluster 0 =  2.3190608479279016e-10
probability of observing that point if it came from cluster 1 =  0.9271355556282294
point =  5.688413532390584
probability of observing that point if it came from cluster 0 =  0.6960729782621278
probability of observing that point if it came from cluster 1 =  3.2658818494280103e-12
point =  5.880260520334039
probability of observing that point if it came from cluster 0 =  0.5095994977687418
probability of observing that point if it came from cluster 1 =  8.358364609447112e-11
point =  6.236036587564709
probability of observing that point if it came from cluster 0 =  0.18816258150877874
probability of observing that point if it came from cluster 1 =  1.9471411838670275e-08
point =  4.330704622709736
probability of observing that point if it came from cluster 0 =  0.06928193578947928
probability of observing that point if it came from cluster 1 =  8.190500525632944e-25
point

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [34]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
        sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0]) * (x[1] - mean[0]) for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x[0] * (x[1] - mean[1]) * (x[1] - mean[1]) for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.405570998307871,  mean_2 = 8.714074949299576
var_1 = 0.4827401410529102,  var_2 = 0.4163550416275781


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [ ]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0]*prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1]*prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [38]:
for i in range(len(data)):
    assigned_cluster = np.argmax([prob_s0_x[i], prob_s1_x[i]])
    print("Point", data[i], "is assigned to cluster", assigned_cluster)

Point 8.607218202703514 is assigned to cluster 1
Point 5.688413532390584 is assigned to cluster 0
Point 5.880260520334039 is assigned to cluster 0
Point 6.236036587564709 is assigned to cluster 0
Point 4.330704622709736 is assigned to cluster 0
Point 8.320755084537545 is assigned to cluster 1
Point 4.892423031457026 is assigned to cluster 0
Point 7.921689632480854 is assigned to cluster 1
Point 9.835974558206726 is assigned to cluster 1
Point 8.884732248835702 is assigned to cluster 1
